<a href="https://colab.research.google.com/github/harry-vaibhav-agarwal/Story/blob/master/SKINCANCER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [34]:
import numpy as np
import pandas as pd
import glob
import matplotlib.pyplot as plt
import skimage.transform
import os
import gc

In [35]:
def get_filelist(img_loc):
    filelist = glob.glob(img_loc+'*.jpg')
    filelist.sort()
    return filelist

In [36]:
img_loc = '/content/drive/MyDrive/MachineLearning/ISIC2018/ISIC2018_Task3_Training_Input/'
filelist =  get_filelist(img_loc)

In [ ]:
filelist

In [38]:
print(len(filelist))

10015


In [39]:
def get_X_orig(filelist, input_shape):
    
    bs, n_H, n_W, n_C = input_shape
    X_orig = np.empty([bs, n_H, n_W, n_C])
 
    for i, fname in enumerate(filelist[:bs]):
        img = plt.imread(fname)
        img = skimage.transform.resize(img, (n_H,n_W), mode='constant')
        X_orig[i] = img
        if i%50 == 49 or i == bs-1:
            print('{} files loaded'.format(i+1))
    
    return X_orig

In [40]:
bs = 1000
input_shape = [bs, 224, 224, 3]
 
X_orig = get_X_orig(filelist, input_shape)

50 files loaded
100 files loaded
150 files loaded
200 files loaded
250 files loaded
300 files loaded
350 files loaded
400 files loaded
450 files loaded
500 files loaded
550 files loaded
600 files loaded
650 files loaded
700 files loaded
750 files loaded
800 files loaded
850 files loaded
900 files loaded
950 files loaded
1000 files loaded


In [41]:
len(X_orig)

1000

In [42]:
def get_Y_orig(labels_loc):
    Y_df = pd.read_csv(labels_loc,nrows=1000)
    Y_orig = np.array(Y_df.iloc[:,1:])
    return Y_orig

In [43]:
labels_loc = '/content/drive/MyDrive/MachineLearning/ISIC2018/ISIC2018_Task3_Training_GroundTruth/ISIC2018_Task3_Training_GroundTruth.csv'
Y_orig = get_Y_orig(labels_loc)

In [44]:
len(Y_orig)

1000

In [45]:
X_orig_file = 'X_orig.npy'
np.save(X_orig_file, X_orig)
 
Y_orig_file = 'Y_orig.npy'
np.save(Y_orig_file, Y_orig)

In [46]:
 
def get_random_crops(filelist, input_shape, Y_orig, num_crops, label):
    
    #rc = random crom
    
    label_dict = {'MEL':0, 'NV':1, 'BCC': 2, 'AKIEC': 3, 'BKL': 4, 'DF': 5, 'VASC': 6}
    label_num = label_dict[label]
    Y_lst = []
    bs, n_H, n_W, n_C = input_shape
    bs_rc = (np.sum((Y_orig[:,label_num]==1).astype(int)))*num_crops
    X_rc = np.empty([bs_rc, n_H, n_W, n_C])
    dfrcloc = 0
    for i, fname in enumerate(filelist[:bs]):
        if Y_orig[i,label_num]:
            img = plt.imread(fname)
            for j in range(num_crops):
                c = np.random.choice(range(-20,20),4)
                img_cropped = img[50+c[0]:400+c[1],50+c[2]:550+c[3],:]
                img_cropped = skimage.transform.resize(img, (n_H,n_W), mode='constant')
                X_rc[dfrcloc] = img_cropped
                Y_lst.append(Y_orig[i])
                dfrcloc +=1
    Y_rc = np.array(Y_lst)
    return X_rc, Y_rc

In [47]:
 
num_crops = 4
 
X_DF_rc, Y_DF_rc = get_random_crops(filelist, input_shape, Y_orig, num_crops, label='DF')

In [48]:
print(X_orig.shape)
print(Y_orig.shape)
print(X_DF_rc.shape)
print(Y_DF_rc.shape)

(1000, 224, 224, 3)
(1000, 7)
(44, 224, 224, 3)
(44, 7)


In [49]:
X_VASC_rc, Y_VASC_rc = get_random_crops(filelist, input_shape, Y_orig, num_crops, label='VASC')

In [51]:
 
print(X_VASC_rc.shape)
print(Y_VASC_rc.shape)

(56, 224, 224, 3)
(56, 7)


In [52]:
def flip_vert(X_orig, Y_orig):
    m, n_H0, n_W0, n_C0 = X_orig.shape
    X_flip = X_orig[:,n_H0-1::-1,:,:]
    Y_flip = Y_orig
    return X_flip, Y_flip

In [53]:
 
X_AKIEC = X_orig[Y_orig[:,3]==1,:]
Y_AKIEC = Y_orig[Y_orig[:,3]==1,:]

In [54]:
X_AKIEC_flip, Y_AKIEC_flip = flip_vert(X_AKIEC, Y_AKIEC)

In [55]:
print(Y_AKIEC.shape)

(36, 7)


In [56]:
 
def flip_hor(X_orig, Y_orig):
    m, n_H0, n_W0, n_C0 = X_orig.shape
    X_flip = X_orig[:,:,n_W0-1::-1,:]
    Y_flip = Y_orig
    return X_flip, Y_flip

In [57]:
X_orig_mirror, Y_orig_mirror = flip_hor(X_orig, Y_orig)
X_VASC_mirror, Y_VASC_mirror = flip_hor(X_VASC_rc, Y_VASC_rc)
X_DF_mirror, Y_DF_mirror = flip_hor(X_DF_rc, Y_DF_rc)
X_AKIEC_flip_mirror, Y_AKIEC_flip_mirror = flip_hor(X_AKIEC_flip, Y_AKIEC_flip)

In [58]:
 
X_with_mirror = np.vstack([X_orig, X_orig_mirror])
Y_with_mirror = np.vstack([Y_orig, Y_orig_mirror])

In [59]:
X_with_mirror_file = 'X_with_mirror.npy'
np.save(X_with_mirror_file, X_with_mirror)
 
Y_with_mirror_file = 'Y_with_mirror.npy'
np.save(Y_with_mirror_file, Y_with_mirror)
#X_orig_inception_file = 'X_orig_inception.npy'
#np.save(X_orig_inception_file, X_orig_inception)

In [60]:
X_final = np.vstack([X_orig, X_DF_rc, X_VASC_rc, X_AKIEC_flip, X_orig_mirror, X_VASC_mirror, X_DF_mirror, X_AKIEC_flip_mirror])
Y_final = np.vstack([Y_orig, Y_DF_rc, Y_VASC_rc, Y_AKIEC_flip, Y_orig_mirror, Y_VASC_mirror, Y_DF_mirror, Y_AKIEC_flip_mirror])

In [61]:
print(X_final.shape)
print(Y_final.shape)

(2272, 224, 224, 3)
(2272, 7)


In [64]:
X_final_file = 'X_final.npy'
np.save(X_final_file, X_final)
 
Y_final_file = 'Y_final.npy'
np.save(Y_final_file, Y_final)
#X_orig_inception_file = 'X_orig_inception.npy'
#np.save(X_orig_inception_file, X_orig_inception)

In [65]:
X_final_file = 'X_final.npy'
X = np.load(X_final_file)

In [66]:
colmins = np.min(np.min(X, axis = 1,keepdims =True), axis = 2, keepdims =True)

In [67]:
colmins.shape

(2272, 1, 1, 3)

In [68]:
X_colcorr = X-colmins

In [70]:
 
X_colcorr[3]

array([[[0.27390456, 0.29850065, 0.28203219],
        [0.31737382, 0.32845795, 0.30653855],
        [0.32120786, 0.33166673, 0.30910708],
        ...,
        [0.34282901, 0.36498256, 0.35619404],
        [0.33897684, 0.36371642, 0.35758897],
        [0.28605724, 0.32796494, 0.31873875]],

       [[0.3007156 , 0.29821366, 0.29359244],
        [0.3663059 , 0.35077031, 0.3437772 ],
        [0.38048657, 0.36237745, 0.36218175],
        ...,
        [0.37539922, 0.40130271, 0.39701662],
        [0.37388643, 0.39588585, 0.37892938],
        [0.32201318, 0.35342293, 0.3359944 ]],

       [[0.29914903, 0.29688375, 0.29627101],
        [0.36395683, 0.34372218, 0.3415338 ],
        [0.37833258, 0.35926058, 0.35234594],
        ...,
        [0.38477579, 0.39730361, 0.39243729],
        [0.38446785, 0.39185924, 0.3857493 ],
        [0.32124287, 0.34756653, 0.33936262]],

       ...,

       [[0.24768189, 0.32057073, 0.25596645],
        [0.30522396, 0.36350509, 0.30134804],
        [0.31250719, 0

In [71]:
colmins[3]

array([[[0.44190489, 0.11213235, 0.18098739]]])